In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, DateType
from pyspark.sql.functions import *

#set up spark (session)
sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("BatchPipeline")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

#set up hadoop fs configuration
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

#retrieve data from bucket
#google storage file path
gsc_file_path = 'gs://de_jads_batch_data/' # bucket name !

dataSchema = StructType(
    [StructField("home_team", StringType(), True),
    StructField("away_team", StringType(), True),
    StructField("home_score", LongType(), True),
    StructField("home_xg", DoubleType(), True),
    StructField("home_penalty", LongType(), True),
    StructField("away_score", LongType(), True),
    StructField("away_xg", DoubleType(), True),
    StructField("away_penalty", LongType(), True),
    StructField("home_manager", StringType(), True),
    StructField("home_captain", StringType(), True),
    StructField("away_manager", StringType(), True),
    StructField("away_captain", StringType(), True),
    StructField("home_goals", StringType(), True),
    StructField("away_goals", StringType(), True),
    StructField("Attendance", LongType(), True),
    StructField("Venue", StringType(), True),
    StructField("Officials", StringType(), True),
    StructField("Date", StringType(), True),
    StructField("Score", StringType(), True),
    StructField("Referee", StringType(), True),
    StructField("Notes", StringType(), True),
    StructField("Round", StringType(), True),
    StructField("Host", StringType(), True),
    StructField("Year", LongType(), True)
    ])

#add all decades to one dataframe
matches = spark.read.format("csv").schema(dataSchema).option("header", "true") \
    .load(gsc_file_path+'*.csv')
    
matches.printSchema()



In [ ]:
#convert StringType to DateType
matches = matches.withColumn("Date", to_date(col("Date"), 'yyyy-MM-dd'))
matches = matches.withColumn("Year", to_date(col("Year"), 'yyyy'))

matches.printSchema()
matches.show(10)

In [ ]:
#write data to bucket
matches.write.format("csv").mode("overwrite").save("gs://de_jads_batch_data/matches.csv") # bucket name !


In [ ]:
#use the cloud storage bucket for temporary BigQuery export data used by the connector
bucket = "de_jads_temp_annelies" # bucket name !
spark.conf.set('temporaryGcsBucket', bucket)

#save the data to BigQuery -> do not forget to change project ID
matches.write.format('bigquery') \
    .option('table', 'de2022-362620.assignment2dataset.matches') \
    .mode("append") \
    .save()

Calculate the total number and average number of goals per year

In [ ]:
from pyspark.sql.functions import *

#nog niet getest of dit werkt + average goals moet nog worden toegevoegd
matches_goals = matches.groupby(col('Year')). \
    agg(sum("home_score" + "away_score").alias("total_goals"))


In [ ]:
# stop the spark context
spark.stop()